In [30]:
import pandas as pd
import numpy as np
import os

In [97]:
file_path = '2020_LER_20200922_testset.xlsx'

# file_path에 따라 변수 결정 (N형만 생각)

if file_path == '2020_LER_20200804_V006.xlsx':
    num_input = 5
    num_output = 6
    num_in_cycle = 50
    num_of_cycle = 100
    num_total = num_in_cycle * num_of_cycle
    num_train = 75
    num_val = 10
    num_test = 15
    x_cols = "B:G"
    y_cols = "H:P"
    header = 0

elif file_path == '2020_LER_20201008_V008.xlsx':
    num_input = 5
    num_output = 6
    num_in_cycle = 50
    num_of_cycle = 127
    num_total = num_in_cycle * num_of_cycle
    num_train = 75
    num_val = 10
    num_test = 0
    x_cols = "D:F"
    y_cols = "H:P"
    header = 0
    
elif file_path == '2020_LER_20200922_testset.xlsx':
    num_input = 5
    num_output = 6
    num_in_cycle = [232, 289, 277, 253, 255]
    num_of_cycle = 5
    num_total = sum(num_in_cycle)
    x_cols = "D:F"
    y_cols = "H:P"
    header = 0

# 2020_LER_20200804_V006

In [75]:
# file_path version : 2020_LER_20200804_V006.xlsx

data_x = pd.read_excel('../'+file_path, sheet_name='Generated DATAs', usecols=x_cols, nrows=2*num_total+1, header=header)
data_y = pd.read_excel('../'+file_path, sheet_name='Generated DATAs', usecols=y_cols, nrows=2*num_total+1, header=header)

# one-hot encoding
X_all , Y_all = np.zeros((num_total, num_input)), np.zeros((num_total, num_output))
X_per_cycle, Y_per_cycle = np.zeros((num_of_cycle, num_input)), np.zeros((num_of_cycle, num_output))

# DATA_X DATA_Y preprocessing
# 1. N, P to 10, 01 (one-hot encoding)
data_x =pd.get_dummies(data_x, columns=['PNMOS'], dtype=float)

# 2. Remove unrequired column ( Wfin [nm], alpha )
# X: Wfin, alpha
data_x = data_x.drop('Wfin [nm]', axis=1)
data_x = data_x.drop('alpha', axis=1)

# Y: IDLO, IDHI, DIBL
data_y = data_y.drop('IDLO', axis=1)
data_y = data_y.drop('IDHI', axis=1)
data_y = data_y.drop('DIBL(mV)', axis=1)

# print(data_y)

# PANDAS TO NUMPY
# X_per_cycle
for i in range(num_of_cycle):
#     print(2*i*num_in_cycle, 2*i*num_in_cycle+1)
    X_per_cycle[i] = data_x[2*i*num_in_cycle:2*i*num_in_cycle+1].values
#     print(X_per_cycle[i])

# X_all
X_all = np.repeat(X_per_cycle,num_in_cycle,axis=0)

# Y_all
for i in range(num_of_cycle):
#     print(i*num_in_cycle,(i+1)*num_in_cycle)
    Y_all[i*num_in_cycle:i*num_in_cycle+num_in_cycle] = data_y[2*i*num_in_cycle:2*i*num_in_cycle + num_in_cycle].values
#     print(Y_all[i*num_in_cycle])

# Y_per_cycle    
for i in range(num_of_cycle):
    Y_per_cycle[i] = np.mean(Y_all[i*num_in_cycle:(i+1)*num_in_cycle],axis=0)

print("============ Data load =============")

print("X data shape: ", X_all.shape, "X per cycle data shape:", X_per_cycle.shape)
print("Y data shape: ", Y_all.shape, "Y per cycle data shape:", Y_per_cycle.shape)  
print("any nan in X?: ", np.argwhere(np.isnan(X_all)))
print("any nan in Y?: ", np.argwhere(np.isnan(Y_all)))

print("============ Data save =============")
data = []
data.append(X_all)
data.append(Y_all)
data.append(X_per_cycle)
data.append(Y_per_cycle)
    
data = np.array(data)
print("saved data shape: ", data.shape)
np.save('./'+file_path, data)

============ Data load =============
X data shape:  (5000, 5) X per cycle data shape: (100, 5)
Y data shape:  (5000, 6) Y per cycle data shape: (100, 6)
any nan in X?:  []
any nan in Y?:  []
============ Data save =============
saved data shape:  (4,)


In [79]:
# 검증의 시간

data = np.load(file_path+'.npy', allow_pickle=True)
    
X_all, Y_all, X_per_cycle, Y_per_cycle = data[0], data[1], data[2], data[3]

print(data[0].shape, data[1].shape, data[2].shape, data[3].shape)

ㅔ갸ㅜ
for i in range(X_per_cycle.shape[0]):
    print(i)
    print(X_per_cycle[i])

(5000, 5) (5000, 6) (100, 5) (100, 6)
0
[  0.19316255  61.00227254 110.61789695   1.           0.        ]
1
[  0.71479321  30.46485839 164.03098759   1.           0.        ]
2
[  0.36639042  45.71598335 128.60254701   1.           0.        ]
3
[ 0.19156989 92.07622363 49.23479694  1.          0.        ]
4
[  0.64566003  28.77580661 181.68354021   1.           0.        ]
5
[  0.24976869  49.66628494 160.81107576   1.           0.        ]
6
[  0.58558021  33.64448087 197.99245715   1.           0.        ]
7
[ 0.6574629  45.59676738 70.67648175  1.          0.        ]
8
[ 0.13748646 47.26343432 23.46686409  1.          0.        ]
9
[  0.76475293  92.24596085 177.48935442   1.           0.        ]
10
[  0.29626603  81.48909977 179.64037297   1.           0.        ]
11
[ 0.6236913  87.47656503 97.32992467  1.          0.        ]
12
[  0.46475666  46.051489   122.05853503   1.           0.        ]
13
[  0.57950422  29.18162993 132.20992165   1.           0.        ]
14
[ 0.19323

# 2020_LER_20201008_V008

In [62]:
# file_path version : 2020_LER_20201008_V006.xlsx

data_x = pd.read_excel('../'+file_path, sheet_name='Generated DATAs', usecols=x_cols, nrows=num_total+1, header=header)
data_y = pd.read_excel('../'+file_path, sheet_name='Generated DATAs', usecols=y_cols, nrows=num_total+1, header=header)

# one-hot encoding (나중에 1, 0 을 인위적으로 넣어주기)
X_all , Y_all = np.zeros((num_total, num_input-2)), np.zeros((num_total, num_output))
X_per_cycle, Y_per_cycle = np.zeros((num_of_cycle, num_input-2)), np.zeros((num_of_cycle, num_output))

# DATA_X DATA_Y preprocessing

# Y: IDLO, IDHI, DIBL
data_y = data_y.drop('IDLO', axis=1)
data_y = data_y.drop('IDHI', axis=1)
data_y = data_y.drop('DIBL(mV)', axis=1)
print(data_x)
# print(data_y)

# PANDAS TO NUMPY
# X_per_cycle
for i in range(num_of_cycle):
#     print(i*num_in_cycle, i*num_in_cycle+1)
    X_per_cycle[i] = data_x[i*num_in_cycle:i*num_in_cycle+1].values
#     print(X_per_cycle[i])

# X_all
X_all = np.repeat(X_per_cycle,num_in_cycle,axis=0)


# 인위적으로 one-hot 넣어주기

X_all = np.hstack((X_all, np.ones((X_all.shape[0], 1))))
X_per_cycle = np.hstack((X_per_cycle, np.ones((X_per_cycle.shape[0], 1))))

X_all = np.hstack((X_all, np.zeros((X_all.shape[0], 1))))
X_per_cycle = np.hstack((X_per_cycle, np.zeros((X_per_cycle.shape[0], 1))))


# Y_all
for i in range(num_of_cycle):
#     print(i*num_in_cycle,(i+1)*num_in_cycle)
    Y_all[i*num_in_cycle:(i+1)*num_in_cycle] = data_y[i*num_in_cycle:(i+1)*num_in_cycle].values
#     print(Y_all[i*num_in_cycle])

# Y_per_cycle    
for i in range(num_of_cycle):
    Y_per_cycle[i] = np.mean(Y_all[i*num_in_cycle:(i+1)*num_in_cycle],axis=0)

print("============ Data load =============")

print("X data shape: ", X_all.shape, "X per cycle data shape:", X_per_cycle.shape)
print("Y data shape: ", Y_all.shape, "Y per cycle data shape:", Y_per_cycle.shape)  
print("any nan in X?: ", np.argwhere(np.isnan(X_all)))
print("any nan in Y?: ", np.argwhere(np.isnan(Y_all)))

print("============ Data save =============")
data = []
data.append(X_all)
data.append(Y_all)
data.append(X_per_cycle)
data.append(Y_per_cycle)
    
data = np.array(data)
print("saved data shape: ", data.shape)
np.save('./'+file_path, data)

          amp.     corr.x      corr.y
0     0.193163  61.002273  110.617897
1          NaN        NaN         NaN
2          NaN        NaN         NaN
3          NaN        NaN         NaN
4          NaN        NaN         NaN
...        ...        ...         ...
6345       NaN        NaN         NaN
6346       NaN        NaN         NaN
6347       NaN        NaN         NaN
6348       NaN        NaN         NaN
6349       NaN        NaN         NaN

[6350 rows x 3 columns]
============ Data load =============
X data shape:  (6350, 5) X per cycle data shape: (127, 5)
Y data shape:  (6350, 6) Y per cycle data shape: (127, 6)
any nan in X?:  []
any nan in Y?:  []
============ Data save =============
saved data shape:  (4,)


In [64]:
# 검증의 시간

data = np.load(file_path+'.npy', allow_pickle=True)
    
X_all, Y_all, X_per_cycle, Y_per_cycle = data[0], data[1], data[2], data[3]

print(data[0].shape, data[1].shape, data[2].shape, data[3].shape)

for i in range(Y_all.shape[0]):
    print(X_per_cycle[i])

(6350, 5) (6350, 6) (127, 5) (127, 6)
[  0.19316255  61.00227254 110.61789695   1.           0.        ]
[  0.71479321  30.46485839 164.03098759   1.           0.        ]
[  0.36639042  45.71598335 128.60254701   1.           0.        ]
[ 0.19156989 92.07622363 49.23479694  1.          0.        ]
[  0.64566003  28.77580661 181.68354021   1.           0.        ]
[  0.24976869  49.66628494 160.81107576   1.           0.        ]
[  0.58558021  33.64448087 197.99245715   1.           0.        ]
[ 0.6574629  45.59676738 70.67648175  1.          0.        ]
[ 0.13748646 47.26343432 23.46686409  1.          0.        ]
[  0.76475293  92.24596085 177.48935442   1.           0.        ]
[  0.29626603  81.48909977 179.64037297   1.           0.        ]
[ 0.6236913  87.47656503 97.32992467  1.          0.        ]
[  0.46475666  46.051489   122.05853503   1.           0.        ]
[  0.57950422  29.18162993 132.20992165   1.           0.        ]
[ 0.19323371 43.90356688 60.43297631  1.    

IndexError: index 127 is out of bounds for axis 0 with size 127

# 2020_LER_0922_testset

In [98]:
# file_path version : 2020_LER_20201008_V006.xlsx

file_path = '2020_LER_20200922_testset.xlsx'

data_x = pd.read_excel('../'+file_path, sheet_name='Generated DATAs', usecols=x_cols, nrows=num_total+1, header=header)
data_y = pd.read_excel('../'+file_path, sheet_name='Generated DATAs', usecols=y_cols, nrows=num_total+1, header=header)

# one-hot encoding (나중에 1, 0 을 인위적으로 넣어주기)
X_all , Y_all = np.zeros((num_total, num_input-2)), np.zeros((num_total, num_output))
X_per_cycle, Y_per_cycle = np.zeros((num_of_cycle, num_input-2)), np.zeros((num_of_cycle, num_output))

# DATA_X DATA_Y preprocessing

# Y: IDLO, IDHI, DIBL
data_y = data_y.drop('IDLO', axis=1)
data_y = data_y.drop('IDHI', axis=1)
data_y = data_y.drop('DIBL(mV)', axis=1)
print(data_x)
# print(data_y)

X_all = data_x.values
Y_all = data_y.values

# X_per_cycle

idx = 0
add = 0
for i in range(num_of_cycle):
    add = num_in_cycle[i]
            
    X_per_cycle[i] = X_all[idx:idx+1]
    Y_per_cycle[i] = np.mean(Y_all[idx:idx+add], axis=0)
            
    temp = X_per_cycle[i].reshape(1, num_input-2)
            
    X_all[idx:idx+add] = np.repeat(temp, add, axis=0)
    idx += add
    
# 인위적으로 one-hot 넣어주기

X_all = np.hstack((X_all, np.ones((X_all.shape[0], 1))))
X_per_cycle = np.hstack((X_per_cycle, np.ones((X_per_cycle.shape[0], 1))))

X_all = np.hstack((X_all, np.zeros((X_all.shape[0], 1))))
X_per_cycle = np.hstack((X_per_cycle, np.zeros((X_per_cycle.shape[0], 1))))

print("============ Data load =============")

print("X data shape: ", X_all.shape, "X per cycle data shape:", X_per_cycle.shape)
print("Y data shape: ", Y_all.shape, "Y per cycle data shape:", Y_per_cycle.shape)  
print("any nan in X?: ", np.argwhere(np.isnan(X_all)))
print("any nan in Y?: ", np.argwhere(np.isnan(Y_all)))  

print("============ Data save =============")
    
data = []
data.append(X_all)
data.append(Y_all)
data.append(X_per_cycle)
data.append(Y_per_cycle)
    
data = np.array(data)
print("saved data shape: ", data.shape)
np.save('./'+file_path, data)

          amp.     corr.x      corr.y
0     0.624847  17.007432  173.857189
1          NaN        NaN         NaN
2          NaN        NaN         NaN
3          NaN        NaN         NaN
4          NaN        NaN         NaN
...        ...        ...         ...
1301       NaN        NaN         NaN
1302       NaN        NaN         NaN
1303       NaN        NaN         NaN
1304       NaN        NaN         NaN
1305       NaN        NaN         NaN

[1306 rows x 3 columns]
============ Data load =============
X data shape:  (1306, 5) X per cycle data shape: (5, 5)
Y data shape:  (1306, 6) Y per cycle data shape: (5, 6)
any nan in X?:  []
any nan in Y?:  []
============ Data save =============
saved data shape:  (4,)


In [99]:
# 검증의 시간

data = np.load(file_path+'.npy', allow_pickle=True)
    
X_all, Y_all, X_per_cycle, Y_per_cycle = data[0], data[1], data[2], data[3]

print(data[0].shape, data[1].shape, data[2].shape, data[3].shape)

for i in range(Y_all.shape[0]):
    print(Y_all[i])

(1306, 5) (1306, 6) (5, 5) (5, 6)
[1.9580e-12 4.2420e-05 1.6180e-05 3.4300e-01 3.5300e-01 6.1501e+01]
[6.1350e-13 3.6300e-05 1.4500e-05 3.7000e-01 3.7700e-01 6.0804e+01]
[7.437e-13 3.182e-05 1.391e-05 3.700e-01 3.740e-01 6.120e+01]
[1.4790e-12 4.1000e-05 1.5990e-05 3.5200e-01 3.5800e-01 6.1429e+01]
[1.9260e-12 4.3930e-05 1.7650e-05 3.4400e-01 3.5300e-01 6.1569e+01]
[1.1990e-12 3.6920e-05 1.4540e-05 3.5600e-01 3.6300e-01 6.1272e+01]
[1.6050e-12 3.6600e-05 1.3990e-05 3.4900e-01 3.5500e-01 6.1348e+01]
[1.5290e-12 4.0250e-05 1.5810e-05 3.5000e-01 3.5700e-01 6.1406e+01]
[2.2580e-12 3.9540e-05 1.6170e-05 3.4200e-01 3.5100e-01 6.1827e+01]
[1.9230e-12 4.2050e-05 1.5870e-05 3.4500e-01 3.5300e-01 6.1627e+01]
[1.1800e-12 4.0380e-05 1.7000e-05 3.5500e-01 3.6300e-01 6.1232e+01]
[2.461e-12 4.101e-05 1.478e-05 3.390e-01 3.490e-01 6.173e+01]
[3.8970e-12 4.2100e-05 1.6900e-05 3.3100e-01 3.4100e-01 6.2348e+01]
[2.6110e-12 4.3680e-05 1.6380e-05 3.3700e-01 3.4800e-01 6.1783e+01]
[2.5380e-12 4.2210e-05 1.6

[1.9460e-12 4.2370e-05 1.6740e-05 3.4400e-01 3.5200e-01 6.1528e+01]
[2.8210e-12 4.3190e-05 1.6510e-05 3.3700e-01 3.4500e-01 6.1912e+01]
[1.2100e-12 4.0100e-05 1.5840e-05 3.5500e-01 3.6100e-01 6.1229e+01]
[1.1900e-12 3.9310e-05 1.5080e-05 3.5500e-01 3.6200e-01 6.1208e+01]
[1.5130e-12 3.9020e-05 1.6100e-05 3.5100e-01 3.5700e-01 6.1536e+01]
[3.7450e-12 4.3600e-05 1.7810e-05 3.3200e-01 3.4100e-01 6.2351e+01]
[4.030e-12 4.368e-05 1.739e-05 3.310e-01 3.410e-01 6.256e+01]
[3.539e-12 4.400e-05 1.725e-05 3.320e-01 3.430e-01 6.221e+01]
[1.4000e-12 4.0690e-05 1.4720e-05 3.5100e-01 3.5900e-01 6.1247e+01]
[2.2210e-12 4.1420e-05 1.7000e-05 3.4300e-01 3.4900e-01 6.1841e+01]
[2.000e-12 4.265e-05 1.691e-05 3.430e-01 3.510e-01 6.155e+01]
[1.9700e-12 4.1760e-05 1.5860e-05 3.4500e-01 3.5200e-01 6.1595e+01]
[2.064e-12 4.240e-05 1.687e-05 3.440e-01 3.510e-01 6.173e+01]
[2.2890e-12 4.0890e-05 1.5820e-05 3.4000e-01 3.4800e-01 6.1602e+01]
[1.2440e-12 3.4430e-05 1.4750e-05 3.5500e-01 3.6100e-01 6.1226e+01]
[1.9

# 검증의 시간

In [92]:
# 혼돈의 시작

file_path1 = '2020_LER_20200922_testset.xlsx'
file_path2 = '2020_LER_20200804_V006.xlsx'

file1 = np.load(file_path1+'.npy', allow_pickle=True)
file2 = np.load(file_path2+'.npy', allow_pickle=True)

X_all_1, Y_all_1, X_per_cycle_1, Y_per_cycle_1 = file1[0], file1[1], file1[2], file1[3]
X_all_2, Y_all_2, X_per_cycle_2, Y_per_cycle_2 = file2[0], file2[1], file2[2], file2[3]

X_per_cycle_2_test = X_per_cycle_2[85:]

new_factor = 2
old_factor = 7

print(X_per_cycle_1)
print(X_per_cycle_2_test.shape)

print(X_per_cycle_1[new_factor])
print(X_per_cycle_2_test[old_factor])




# for i in range(5):
#     print(X_per_cycle_1[i])
    
# # print(X_per_cycle_2_test)
# for j in range(15):
#     print(X_per_cycle_2_test[i])

[[  0.62484672  17.00743225 173.85718891   1.           0.        ]
 [  0.55654727  80.91905766  80.56986979   1.           0.        ]
 [  0.50466944  48.62398816  67.99380535   1.           0.        ]
 [  0.56045148  25.42868151  69.39740975   1.           0.        ]
 [  0.63730798  69.14860261  65.33037708   1.           0.        ]]
(15, 5)
[ 0.50466944 48.62398816 67.99380535  1.          0.        ]
[ 0.50466944 48.62398816 67.99380535  1.          0.        ]
